In [1]:
install.packages(c("stringr","stringdist","dplyr","readxl"))

Installing packages into ‘/home/User1/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)

also installing the dependencies ‘lifecycle’, ‘purrr’, ‘rematch’, ‘hms’, ‘prettyunits’, ‘magrittr’, ‘stringi’, ‘pkgconfig’, ‘R6’, ‘tibble’, ‘tidyselect’, ‘BH’, ‘plogr’, ‘cellranger’, ‘progress’




In [1]:
library(stringr)
library(stringdist)
library(dplyr)
library(readxl)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
path <- '/home/User1/data/self_citations/'

In [81]:
col_status <- rep("NULL",7)
col_status[c(4,7)] <- "character"
col_status[1] <- 'integer'
df_authors <- read.csv(paste0(path,"clusters_locations.txt"),sep="\t",colClasses =col_status)

In [3]:
df_inst <- unique(df_authors[,c(2,3)])
#df_inst$Institution <- tolower(df_inst$Institution)

In [4]:
#Map author's institution to Leid institution with fuzzy matching and export
file_name_lei <- paste0(path,'model/Leiden_ranking.xlsx')
df_leiden_raw <- read_excel(file_name_lei,sheet=2, col_types = "text")
var_leiden <- c("University","Country","Field","Period","PP_top5")


fields <- unique(df_leiden_raw$Field)

df_leiden <- df_leiden_raw %>%
  subset(Frac_counting==1 & Period == "2006–2009" & Field == "All sciences")
df_leiden$University <- iconv(df_leiden$University,from="UTF-8",to="ASCII//TRANSLIT")
df_leiden <- df_leiden[,var_leiden]
rm(df_leiden_raw)


In [95]:
write.csv(df_leiden,paste0(path,'leid_prop5.csv'),row.names = FALSE)

In [76]:
#df_leiden$Country <- tolower(df_leiden$Country)

In [5]:
df_leiden <- data.frame(df_leiden)

In [7]:
nb_comp <- nrow(df_inst)

In [94]:
head(df_leiden)

,University,Country,Field,Period,PP_top5
,<chr>,<chr>,<chr>,<chr>,<chr>
1,aalborg university,denmark,All sciences,2006–2009,3.9527131084826E-2
2,aalto university,finland,All sciences,2006–2009,4.7843169987152698E-2
3,aarhus university,denmark,All sciences,2006–2009,6.2751106656845898E-2
4,adam mickiewicz university in poznan,poland,All sciences,2006–2009,1.41971144761704E-2
5,agh university of science and technology,poland,All sciences,2006–2009,9.8638073842913804E-3
6,ain shams university,egypt,All sciences,2006–2009,2.7408554772684699E-2


In [16]:
#leid_inst <- unique(df_leiden$University)
leid_country <- unique(df_leiden$Country)
leid_country_lo <- tolower(leid_country)

In [21]:
all_inst <- data.frame(inst=character(),
                       leid_inst=character(),
                       country=character(),
                       leid_country=character(),
                       dist_inst=double(),stringsAsFactors=FALSE)

In [22]:
nb_comp

[1] 1753827

In [ ]:

for(i in 1573370:nb_comp){
    inst <- df_inst[i,"Institution"]
    country <- df_inst[i,"ERegroupement"]
    inst_lo <- tolower(inst)
    country_lo <- tolower(country)
    if(!is.na(inst) & nchar(inst)>1 & !is.na(country) & nchar(country)>1){
        
        dist_country <- lapply(leid_country_lo,function(x){stringdist(country_lo,x,method='cosine',q=2)})
        min_country <- Reduce(min,dist_country)
        argmin_country <- which.min(unlist(dist_country))
        country_min <- leid_country[argmin_country]
        
        leid_inst <- df_leiden[df_leiden$Country==country_min,"University"]
        dist_inst <- lapply(leid_inst,function(x){stringdist(inst_lo,tolower(x),method='cosine',q=2)})
        min_inst <- Reduce(min,dist_inst)
        argmin_inst <- which.min(unlist(dist_inst))
        
        inst_min <- leid_inst[argmin_inst]  
        all_inst[i,] <- c(inst,inst_min,country,country_min,min_inst)
            
        
    }
}

In [111]:
new_DF <- all_inst[all_inst['dist_inst'] == 'NA',]

In [112]:
head(new_DF)

inst,leid_inst,country,leid_country,dist_inst
<chr>,<chr>,<chr>,<chr>,<chr>


In [105]:
all_inst <- all_inst[rowSums(is.na(all_inst)) == 0,]

In [108]:
new_DF <- all_inst[rowSums(is.na(all_inst)) > 0,]

In [110]:
all_inst[154553:154556,]

,inst,leid_inst,country,leid_country,dist_inst
,<chr>,<chr>,<chr>,<chr>,<chr>
447019,RIVERSIDE-REG-CTY-MED-CTR,"University of California, Riverside",United States,United States,0.609905251197253
452163,WHITE-RIVER-JUNCTION-VAH,Ohio University,United States,United States,0.609905251197253
473521,MONTREAL-CRCHUM,Universite de Montreal,Canada,Canada,0.609905251197253
478732,PARAMADINA-UNIV,Banaras Hindu University,Indonesia,India,0.609905251197253


In [106]:
head(all_inst)

,inst,leid_inst,country,leid_country,dist_inst
,<chr>,<chr>,<chr>,<chr>,<chr>
540975,LOUGHBOROUGHUNIVERSITY,Loughborough University,United Kingdom,United Kingdom,0.0543891423106997
303772,KAROLINSKAINSTITUTET,Karolinska Institutet,Sweden,Sweden,0.0586425513367167
65424,NANCHANGUNIVERSITY,Nanchang University,China,China,0.0766194831233614
183569,YEUNGNAMUNIVERSITY,Yeungnam University,South Korea,South Korea,0.0766194831233614
485656,TSINGHUAUNIVERSITY,Tsinghua University,China,China,0.0766194831233614
228592,KAROLINSKA-INSTITUTET,Karolinska Institutet,Sweden,Sweden,0.0769230769230768


In [113]:
write.csv(all_inst,paste0(path,'merged_leid_inst.csv'),row.names = FALSE)

In [37]:
all_inst <- all_inst[order(all_inst$dist_inst),]

In [64]:
all_inst_compl <- all_inst[complete.cases(all_inst),]

In [116]:
options(repr.matrix.max.rows=100, repr.matrix.max.cols=200)

In [40]:
head(all_inst[all_inst$dist_inst>=0.48,],100)

,inst,leid_inst,country,leid_country,dist_inst
,<chr>,<chr>,<chr>,<chr>,<chr>
124103,UNIV-PITTSBURGH-HLTH-SCI,University of Pittsburgh,United States,United States,0.48
149631,JOURNAL-NANJING-MED-UNIV,Nanjing Medical University,China,China,0.48
170196,UNIV-MANITOBA-&-HLTH-SCI,University of Manitoba,Canada,Canada,0.48
214608,UNIV-PITTSBURGH-SHCOOL-MED,University of Pittsburgh,United States,United States,0.48
223707,BUENOS-AIRES-UNIV-HOSP,University of Buenos Aires,Argentina,Argentina,0.48
260733,SCHWARZ-EB-UNIV-PITTSBURGH,University of Pittsburgh,United States,United States,0.48
397191,UNIV-VIRGINIA-HEATH-SYST,West Virginia University,United States,United States,0.48
403980,W-VIRGINIA-UNIV-RES-CORP,West Virginia University,United States,United States,0.48
406411,SUN-YAT-SEN-UNIV-CHINA,Sun Yat-sen University,China,China,0.48


In [42]:
threshold <- 0.5

In [66]:
all_inst_thresh <- all_inst[all_inst_compl$dist_inst<threshold,]

In [86]:
df_matched <- merge(df_authors,all_inst_thresh[,c('inst','country','leid_inst')],by.x=c('Institution','ERegroupement'),by.y=c('inst','country'))

In [102]:
head(new_DF)

Institution,ERegroupement,Cluster_ID,leid_inst
<chr>,<chr>,<int>,<chr>


In [90]:
write.csv(df_matched,paste0(path,'matched_leid_inst.csv'),row.names = FALSE)

In [92]:
df_matched <- merge(df_matched,df_leiden[,c("Country","University","PP_top5")],by.x=c("ERegroupement","leid_inst"),by.y=c("Country","University"))

In [85]:
length(unique(df_authors$Cluster_ID))

[1] 22166707

In [89]:
head(df_matched)

,Institution,ERegroupement,Cluster_ID,leid_inst
,<chr>,<chr>,<int>,<chr>
1,00014-UNIV-HELSINKI,Finland,34160162,University of Helsinki
2,00014-UNIV-HELSINKI,Finland,22212131,University of Helsinki
3,00014-UNIV-HELSINKI,Finland,41363395,University of Helsinki
4,00014-UNIV-HELSINKI,Finland,9022469,University of Helsinki
5,00014-UNIV-HELSINKI,Finland,41363395,University of Helsinki
6,00014-UNIV-HELSINKI,Finland,23390018,University of Helsinki


In [91]:
head(df_leiden)

,University,Country,Field,Period,PP_top5
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Aalborg University,Denmark,All sciences,2006–2009,3.9527131084826E-2
2,Aalto University,Finland,All sciences,2006–2009,4.7843169987152698E-2
3,Aarhus University,Denmark,All sciences,2006–2009,6.2751106656845898E-2
4,Adam Mickiewicz University in Poznan,Poland,All sciences,2006–2009,1.41971144761704E-2
5,AGH University of Science and Technology,Poland,All sciences,2006–2009,9.8638073842913804E-3
6,Ain Shams University,Egypt,All sciences,2006–2009,2.7408554772684699E-2
